In [3]:
## import all necessary packages
import json
import re
import pandas as pd

import nltk, string
from nltk.collocations import *

import numpy as np
import matplotlib.pyplot as plt

from pandas import *

from collections import defaultdict

import string
import happyfuntokenizing
import json

In [4]:
with open("./data/categories_manual.json") as json_file:
    category_dict = json.load(json_file)

In [5]:
with open('./data/emoji_webscraped_expanded.json','r') as f_emoji:
    emoji_df = DataFrame(json.load(f_emoji))

In [6]:
with open('./data/tweets_test_clean.json','r') as f:
    tweet_df = DataFrame(json.load(f))

In [7]:
tweet_df.head()

,id,only_HashTag,only_emoji,only_text_splithashtag,retweet,split_hashtag,text,user_id
0,377652254096228352,[],[],hdl hey checkout the website : url,None,[],hdl hey checkout the website: url,224874450
1,378043241884422144,[],[],hdl hahahah please do it !,None,[],hdl hahahah please do it!,240974461
10,378990150476255232,[],[],In one night my mom made the same that I made working one month .,None,[],In one night my mom made the same that I made working one month.,72715638
100,378992761594056704,[],[😆],"My cousin , aunt , and uncle came to visit my house just to watch the $ 70 boxing match that my dad pay per viewed . Lol .",None,[],"My cousin, aunt, and uncle came to visit my house just to watch the $70 boxing match that my dad pay per viewed. Lol. 😆",42533846
1000,379024319990349825,[],[😑],Typical,None,[],Typical 😑,1113487506


## Statistics on all emojis

**emoji_array**:  list of all emojis

In [8]:
emoji_array = [word for item in tweet_df.only_emoji for word in item]
len(emoji_array)

74671

**unique_emoji**:  number of unique emojis

In [9]:
unique_emoji = set(emoji_array)
len(unique_emoji)

663

**full_dict**: percentage of emojis by emoji

In [10]:
full_dict = defaultdict(int)
for item in emoji_array:
    full_dict[item] += 1

In [11]:
for item in sorted(full_dict.items(), key=lambda x:x[1], reverse=True)[:50]:
    print(item[0], (float(item[1])/len(emoji_array))*100)

😂 12.49213215304469
😍 5.356831969573195
😃 4.4006374630043785
😭 4.235914879940003
❤ 3.157852446063398
😘 2.4520898340721295
👌 2.178891403623897
😞 1.8856048532897642
😏 1.825340493632066
😊 1.7342743501493216
😒 1.6940981103775228
😁 1.6874020704155563
😩 1.6793668224611966
💕 1.6003535509099918
👏 1.5816046390164855
😉 1.5400891912522934
😔 1.4771464156098084
🙌 1.3566176962944114
☺ 1.3392079923932987
😳 1.2976925446291063
🎶 0.9186966827818029
🙏 0.8892341069491503
😕 0.8731636110404306
👍 0.8463794511925647
😋 0.8356657872534183
🎉 0.7700445956261467
✌ 0.7379036038087076
💁 0.7204938999075946
😑 0.6816568681281889
😌 0.6575511242651096
😎 0.6334453804020302
😡 0.6267493404400638
🙈 0.6039828045693777
💛 0.578537852713905
😛 0.5705026047595452
😴 0.551753692866039
😀 0.5423792369192859
😫 0.5330047809725329
😷 0.5209519090409931
😱 0.5169342850638132
😜 0.5048814131322736
💙 0.48211487726158747
💜 0.47006200533004777
😢 0.46068754938329476
🔫 0.46068754938329476
♥ 0.45131309343654163
💋 0.43256418154303544
🙊 0.42854655756

## Statistics on only faces

**unique_face**: unique faces from category_dict file

In [12]:
unique_face = [face for item in category_dict.values() for face in item]
len(unique_face)

80

**all_faces**: from emoji_array (list of all emojis), subset by unique_face

In [13]:
all_faces = [unique for face in emoji_array for unique in unique_face if face in unique]
len(all_faces)

45924

In [14]:
all_faces_unique = set(all_faces)
len(all_faces_unique)

80

**categorize**: list of all_faces defined by their respective category from cateogry_dict

In [15]:
cateogrize = []
for category, faces in category_dict.items():
    for every_face in all_faces:
            if every_face in faces:
                cateogrize.append((category, every_face))

In [16]:
# cateogrize

**categorize_percentage**: dictionary with count of each categroy form categorize

In [17]:
category_percentage = defaultdict(int)
for item in cateogrize:
    category_percentage[item[0]] += 1

In [18]:
for item in sorted(category_percentage.items(), key=lambda x:x[1], reverse=True)[:50]:
    print(item[0], (float(item[1])/len(cateogrize))*100)

happy 48.96132741050431
despairing 35.18639491333507
love 13.267572511105305
nonhuman 2.5847051650553086


## Add category column to dataframe

In [19]:
def cateogry_column(text):
    if len(text) == 0:
        return None
    
    boolean_face = [True if face in unique_face else False for face in text]
#     print(boolean_face)
    
    for item in boolean_face:
        if all(boolean_face) == True:
        
            cateogrize = []
            for category, faces in category_dict.items():
                for face in text:
                        if face in faces:
                            cateogrize.append(category)
            cateogrize = list(set(cateogrize))
            if len(cateogrize)==1:
                return cateogrize[0]
            else:
                return "Mix_P"
                
            return list(set(cateogrize))
        
        elif all(not element for element in boolean_face):
            return "Other"
#         else:
#             return "Mix_P_NonP"
        else:
            true_boolean = [index for index, item in enumerate(boolean_face) if item == True]
            keep_true = [text[i] for i in true_boolean]
            
            cateogrize = []
            for category, faces in category_dict.items():
                for face in keep_true:
                        if face in faces:
                            cateogrize.append(category)
            cateogrize = list(set(cateogrize))
            if len(cateogrize)==1:
                return cateogrize[0]
            else:
                return "Mix_P"

    return

In [20]:
print(cateogry_column([]))
print(cateogry_column(["❤"]))
print(cateogry_column(["👌"]))
print(cateogry_column(["😒", "😀"]))
print(cateogry_column(["😒", "👌"]))
print(cateogry_column(["😒","❤","😩", "👌"]))

None
Other
Other
Mix_P
despairing
despairing


In [21]:
tweet_df["cateogry_prediction"] = tweet_df.only_emoji.apply(cateogry_column)

In [22]:
dict_category_prediction = defaultdict(int)
for item in tweet_df.cateogry_prediction:
    dict_category_prediction[item] += 1

In [23]:
dict_category_prediction

defaultdict(<class 'int'>, {'despairing': 9643, 'Mix_P': 2204, None: 156283, 'happy': 13244, 'nonhuman': 614, 'Other': 10310, 'love': 2832})

In [24]:
(11559 + 52558 + 38702 + 2345) / (len(tweet_df) - 625122)

-0.24457199203706115

In [25]:
(11559 + 52558 + 38702 + 2345)

105164

## Make Final column numeric

In [26]:
tweet_df.head()

,id,only_HashTag,only_emoji,only_text_splithashtag,retweet,split_hashtag,text,user_id,cateogry_prediction
0,377652254096228352,[],[],hdl hey checkout the website : url,None,[],hdl hey checkout the website: url,224874450,None
1,378043241884422144,[],[],hdl hahahah please do it !,None,[],hdl hahahah please do it!,240974461,None
10,378990150476255232,[],[],In one night my mom made the same that I made working one month .,None,[],In one night my mom made the same that I made working one month.,72715638,None
100,378992761594056704,[],[😆],"My cousin , aunt , and uncle came to visit my house just to watch the $ 70 boxing match that my dad pay per viewed . Lol .",None,[],"My cousin, aunt, and uncle came to visit my house just to watch the $70 boxing match that my dad pay per viewed. Lol. 😆",42533846,happy
1000,379024319990349825,[],[😑],Typical,None,[],Typical 😑,1113487506,despairing


In [27]:
numeric_category = [(index, value) for index, value in enumerate(list(set(tweet_df.cateogry_prediction)))]
numeric_category

[(0, 'happy'),
 (1, 'love'),
 (2, 'Other'),
 (3, 'despairing'),
 (4, 'Mix_P'),
 (5, 'nonhuman'),
 (6, None)]

In [28]:
def make_numeric(text):
    _num = [item[0] for item in numeric_category if text == item[1]]
    return _num[0]

In [29]:
make_numeric(None)

6

In [30]:
tweet_df['category_numeric'] = tweet_df.cateogry_prediction.apply(make_numeric)

In [31]:
tweet_df.shape

(195130, 10)

In [32]:
tweet_df.to_json('./data/tweets_test_clean_preprocessing.json', force_ascii=False)

In [33]:
tweet_df

,id,only_HashTag,only_emoji,only_text_splithashtag,retweet,split_hashtag,text,user_id,cateogry_prediction,category_numeric
0,377652254096228352,[],[],hdl hey checkout the website : url,None,[],hdl hey checkout the website: url,224874450,None,6
1,378043241884422144,[],[],hdl hahahah please do it !,None,[],hdl hahahah please do it!,240974461,None,6
10,378990150476255232,[],[],In one night my mom made the same that I made working one month .,None,[],In one night my mom made the same that I made working one month.,72715638,None,6
100,378992761594056704,[],[😆],"My cousin , aunt , and uncle came to visit my house just to watch the $ 70 boxing match that my dad pay per viewed . Lol .",None,[],"My cousin, aunt, and uncle came to visit my house just to watch the $70 boxing match that my dad pay per viewed. Lol. 😆",42533846,happy,0
1000,379024319990349825,[],[😑],Typical,None,[],Typical 😑,1113487506,despairing,3
10000,379365568547409920,[],[],Spring Breakers Forever . @ Shoreline Amphitheatre url,None,[],Spring Breakers Forever. @ Shoreline Amphitheatre url,77613655,None,6
100000,382671627341025281,"[#celebrate2013, #ucdavis]",[],The mass exodus of new students outside following the New Student Convocation . celebrate 2013 ucd avis … url,None,"[[celebrate, 2013], [ucd, avis]]",The mass exodus of new students outside following the New Student Convocation. #celebrate2013 #ucdavis… url,19638500,None,6
100001,382671650179014656,[#tracksts],[],Nearly showtime ... tracks ts @ Skyline Drive In Theater url,None,"[[tracks, ts]]",Nearly showtime...\n#tracksts @ Skyline Drive In Theater url,4310571,None,6
100002,382671652540006400,[],[],7:00 PM @ Garlic City Casino $ 20 No-Limit Hold'em w / Rebuys & amp ; Add-on $ 1K Guarantee hdl url,None,[],7:00 PM @ Garlic City Casino $20 No-Limit Hold'em w/ Rebuys &amp; Add-on $1K Guarantee hdl url,132687229,None,6
100003,378552331660312576,[#bruinpride],"[🐻, 💙]",Good luck to my boys playin for Branham today ! bruin pride,None,"[[bruin, pride]]",Good luck to my boys playin for Branham today! #bruinpride 🐻💙,327291147,nonhuman,5
